In [1]:
import time
import requests
from bs4 import BeautifulSoup
import json
import re
import time
import faker
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from dateutil import parser

In [2]:
fake = faker.Faker(locale='en')

In [3]:
amazon_headers = {
    'User-Agent': fake.chrome(),
    'accept-language': 'en-US,en;q=0.9',
    'pragma': 'np-cache',
    'Host': 'www.amazon.jobs',
    'content-type': 'application/json',
    'referer': 'https://www.amazon.jobs/en/search?base_query=&loc_query='
}

In [4]:
clear_string = lambda x: re.sub(' +', ' ', re.sub('<.*?>', ' ', x).replace('\n', '\n ')).strip()

Здесь выбраны только FullTime и Part Time вакансии отсортированные по дате добавления (сначала более новые)

In [5]:
API_URL = "http://www.amazon.jobs/en/search.json?schedule_type_id[]=Flex-Time (<19 hours)&schedule_type_id[]=Part-Time (20-29 hours)&schedule_type_id[]=Reduced-Time (30-39 hours)&schedule_type_id[]=Flex Time (<19 hours)&schedule_type_id[]=Reduced Time (30-39 hours)&schedule_type_id[]=Full-Time&radius=24km&facets[]=normalized_country_code&facets[]=normalized_state_name&facets[]=normalized_city_name&facets[]=location&facets[]=business_category&facets[]=category&facets[]=schedule_type_id&facets[]=employee_class&facets[]=normalized_location&facets[]=job_function_id&facets[]=is_manager&facets[]=is_intern&sort=recent&latitude=&longitude=&loc_group_id=&loc_query=&base_query=&city=&country=&region=&county=&query_options=&"
MAIN_URL = "https://www.amazon.jobs"

### Тестирование для одной страницы

In [6]:
search_params = {
    'offset': 0,
    'result_limit': 100
}

In [7]:
result = requests.get(API_URL, headers=amazon_headers, params=search_params)
result

<Response [200]>

In [8]:
result.json()['jobs'][1]

{'basic_qualifications': '<table style="width: 203px;"><colgroup><col></col></colgroup><tbody><tr><td colspan="1" rowspan="1" class="xl67" style="height: 18px; width: 203px; white-space: nowrap;"><span style="font-size: 13px;"><strong><span style=""><span style="color: black;"><span style="font-style: normal;"><span style="text-decoration: none;">Requisitos básicos:</span></span></span></span></strong></span></td></tr><tr><td colspan="1" rowspan="1" class="xl66" style="height: 18px; white-space: nowrap;"><span style="font-size: 13px;"><span style=""><span style="color: black;"><span style="font-weight: 400;"><span style="font-style: normal;"><span style="text-decoration: none;">· Ser mayor de 18 años.</span></span></span></span></span></span></td></tr><tr><td colspan="1" rowspan="1" class="xl66" style="height: 18px; white-space: nowrap;"><span style="font-size: 13px;"><span style=""><span style="color: black;"><span style="font-weight: 400;"><span style="font-style: normal;"><span styl

### Все страницы

In [9]:
job_dicts = []
for page in tqdm(range(0, 100)): # max offset for amazon is 9999 :(
    search_params = {
        'offset': page * 100,
        'result_limit': 100
    }
    result = requests.get(API_URL, headers=amazon_headers, params=search_params).json()
    if result['jobs']:

        for job_info in result['jobs']:
            try:
                job_dict = {
                    'title': job_info['title'],
                    'internal_id' : job_info['id'],
                    'url': MAIN_URL + job_info['job_path'],
                    'description': clear_string(job_info['description']),
                    'responsibilities': None,
                    'qualifications': clear_string(job_info['basic_qualifications']),
                    'team': " ".join(job_info['team']['label'].split('-')) if job_info['team']['label'] else None,
                    'company': 'Amazon', #job_info['company_name'],
                    'publish_date': parser.parse(job_info['posted_date']),
                }
                job_dicts.append(job_dict)
            except:
                print(f'Error while parsing {MAIN_URL + job_info["job_path"]}')

    time.sleep(0.2) # sleep for 200 ms

100%|██████████| 100/100 [02:54<00:00,  1.75s/it]


In [10]:
snapshot = pd.DataFrame(job_dicts)
snapshot.sample(5)

,title,internal_id,url,description,responsibilities,qualifications,team,company,publish_date
3505,"Enterprise Account Manager, Enterprise Greenfi...",a714f80e-f79f-44a2-af6f-ee4347d41d1b,https://www.amazon.jobs/en/jobs/2179992/enterp...,"Job summary According to an IDC 2021 Report, i...",None,Extensive experience as a quota carrying techn...,team sales primary,Amazon,2022-08-03
2601,"HR Regional Partner, SPS, Selling Partner Serv...",7400ea74-799e-499a-b8d0-6e3aa1bda139,https://www.amazon.jobs/en/jobs/2182124/hr-reg...,Job summary We are looking for a highly motiva...,None,"Bachelor’s degree or higher, in Human Resource...",team hr for hr,Amazon,2022-08-04
6367,"Director, Product Management",4ea94e13-eb73-4119-bf4f-90891ab33d9f,https://www.amazon.jobs/en/jobs/2171442/direct...,Job summary Good storytelling starts with grea...,None,5+ years of Product Management experience. Exp...,team global audible,Amazon,2022-07-27
7192,"BD Spec, Disaster Recovery, AWS Disaster Recovery",4dc4737c-54fc-4152-b686-2c3d5c283d11,https://www.amazon.jobs/en/jobs/2169272/bd-spe...,Job summary The Go-To-Market Cloud Foundations...,None,7+ years of sales experience for DR / Storage ...,team sales primary,Amazon,2022-07-26
6300,"Software Development Engineer , Intech",1386a99f-d516-4dda-b519-b2513dc7f13f,https://www.amazon.jobs/en/jobs/2171592/softwa...,Job summary Você já se perguntou como a Amazon...,None,Proficiência na língua inglesa (nível técnico)...,no team listed,Amazon,2022-07-27


In [11]:
snapshot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   title             10000 non-null  object        
 1   internal_id       9982 non-null   object        
 2   url               10000 non-null  object        
 3   description       10000 non-null  object        
 4   responsibilities  0 non-null      object        
 5   qualifications    10000 non-null  object        
 6   team              9958 non-null   object        
 7   company           10000 non-null  object        
 8   publish_date      10000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(8)
memory usage: 703.2+ KB


In [13]:
current_date = datetime.now().strftime('%d-%m-%Y')
current_date

'10-08-2022'

In [14]:
snapshot.to_csv(f'../data/amazon/{current_date}.csv')
snapshot.to_excel(f'../data/amazon/{current_date}.xlsx', engine='xlsxwriter')